<small><i>This notebook was put together by [Abel Meneses-Abad](http://www.menesesabad.com) for SciPy LA Habana 2017. Source and license info is on [github repository](http://github.com/sorice/simtext_scipyla2017).</i></small>

# Sentence Alignment Algorithm Phase

The objective of this phase is to get a <font color='#FA0000'>new text structure</font> with normalized sentences and its original related by the offset and length properties. E.g.:

<a id='Aligned_Text_Structure'></a>
<font color='#FA0000'>**Aligned Text Structure:**</font>
If you open a the abstract-name file *suspicious-document00XYZ.txt*, then you would get a text with de following structure in every line:

<p><font color='#FA0000'>
   $(id_K,normalized\ sentence_K,original\_offset_{sentence\,K},original\_offset+length_{sentence\,K})$
   </font>

*Why we need this?*
This is useful for example when you are working in a real plagiarism detection or text-reuse detection applications and you need to show to the users the similarity result between the preprocessed fragments or sentences and its original form (pdf, html, etc). Additionally is very safe to work with duplicated or transformed objects very well related to the originals, after have them you can do subsequent transformations to the copy-object and never loss the original length and position of this sentence or fragment.

# Alignment Algorithm based on Jaccard Distance

The final algorithm needs 4 auxiliar functions:

1. A character replacement function in the original text in which Jaccard (and others) fails (*Eg: line breaks*) (**normalize**)
2. A function to look over the list of sentences of the preproces text (**getSentA**).
3. A function to look over the list of segments of the original-text ending with '.'(**getSentB**).
4. Jaccard function to validate Smith-Watermna alignment (**Jaccard**).

In [1]:
#Import cell
import re
import sys

## Original Text Normalization Function

To improve the precision of Jaccard algorithm is necessary to convert some punctuation situation.
The main feature here is that whole regular expressions and functiones used does not change the length of the original text. The [previous](02.1-Normalizing-Text-Corpus.ipynb) normalization process it doesn't.

In [17]:
from preprocess.normalize import  add_doc_ending_point, abbreviations
from preprocess.normalize import  multipart_words, replace_point_sequence


def normalize(text_orig):
    replacement_patterns = [(r'[:](?=\s*?\n)','##1'),
                            (r'\xc2|\xa0',' '),
                            (r'(\w\s*?):(?=\s+?[A-Z]+?)|(\w\s*?):(?=\s*?"+?[A-Z]+?)','\g<1>##2'),
                            (r'[?!]','##3'),
                            # any alphanumeric char follow by \n then by punct-sign then by Capital new sent
                            (r'(\w+?)(\n)(?=["$%()*+&,-/;:¿¡<=>@[\\]^`{|}~\t\s]*(?=.*[A-Z0-9]))','\g<1>##4'), 
                            # alphanum follow by \n follow by any number of point-sign 
                            #follow by a capital letter, replace by alphanumerig+.
                            (r'(\w+?)(\n)(?=["$%()*+&,-/;:¿¡<=>@[\\]^`{|}~\t\s\n]*(?=[a-zA-Z0-9]))','\g<1>##5'),
                            (r'[:](?=\s*?)(?=["$%()*+&,-/;:¿¡<=>@[\\]^`{|}~\t\s]*[A-Z]+?)','##6'),
                            (r'(\w+?\s*?)\|','\g<1>##7'),
                            (r'\n(?=\s*?[A-Z]+?)','##8'),
                            (r'[.](?=\s*?\n+?[a-z]+?)', '##8'),
                            (r'[.](?=\s+?[a-z]+?)', '_'),
                            (r'##\d','apdbx'),
                            ]
    
    for (pattern, repl) in replacement_patterns:
            (text_orig, count) = re.subn(pattern, repl, text_orig)
    
    text_orig = replace_point_sequence(text_orig)
    text_orig = multipart_words(text_orig)
    text_orig = abbreviations(text_orig)
    text_orig = re.sub(r'apdbx+','.', text_orig)
    text_orig = add_doc_ending_point(text_orig)#append . final si el último caracter no tiene punto, evita un ciclo infinito al final.
    return text_orig

This minimal normalization function has been created only with the purpouse to delete some troublesome strings without any lenght modification, the normalization demo flow used in preprocess library change the lenght of all texts.

In [20]:
orig_text = open('data/orig/src/source-document00534.txt').read()
print(len(orig_text))
norm_text = normalize(orig_text)
print(len(norm_text))

12237
12237


## Get Sentence A

Get all sentences in preprocessed text. 

In [4]:
def getSentA(text):
    offset = 0
    for i in re.finditer('\.',text):
        sentA = text[offset:i.end()]
        yield sentA, offset, i.end()
        offset = i.end()+1

In [5]:
#Example of getSentA use
preprocessed_text = open('data/norm/susp/suspicious-document00184.txt').read()
        
for i,(sentA, offset, length) in enumerate(getSentA(preprocessed_text)):
    print (i,sentA, offset, length)

0 The is a 5 acre 20 000 m located in the of located in . 0 55
1 The zoo is operated by the in partnership with the . 56 108
2 Queens Zoo zoo New York City borough Queens Flushing Meadows_Corona Park Wildlife Conservation Society New York City Department of Parks and Recreation . 109 262
3 The Zoo opened on with the ceremonial ribbon cut by . 263 316
4 The zoo is home mostly to animals native to North America . 317 376
5 The Queens Zoo is the only one of the five zoos in New York City to exhibit . 377 454
6 The zoo was constructed on the site of the and the zoo is aviary is a designed by and used during the 1964 Fair . 455 568
7 October 26 1968 Robert Moses 2 Spectacled Bears 1964 New York World is Fair geodesic dome Buckminster Fuller 1  . 569 682


## Get Sentence B

<center><strong>Diagrama simplificado getSentB function.</strong></center></br>
<table border=0 cellspacing=10> 
    <caption align="bottom"> </br><em>Figura 2.3.1: Esquema del Algoritmo Get Sentence B</em>
    </caption> 
<tr align="center">
    <th> <img src="imgs/getSentB.jpg" height=800px width=1200px alt="*" 
        align="center"> </th>
    <th> </th>
    <td> 
        <p> El algoritmo comienza a buscar en el texto original el caracter '.'  más cercano al offsetB (*$\|\overrightarrow{sB}\|=0 \Longrightarrow sentLength = 0$*).
        Una vez encontrado denota como punto previo este nuevo punto para una nueva llamada (*$sentLength = \|\overrightarrow{sB}\|+1$*), 
        luego calcula el segmento, y define el punto siguiente como $Offset + \|\overrightarrow{sB}\|$ *(len segmento definido por el . encontrado)*.
        En la siguiente corrida el algoritmo encontrará el '.' más cercano a partir de **sB** 
        calculando el nuevo punto siguiente(*sB`*). Y así sucesivamente
        ante cada llamada en la función *getSentB*. Es en la función de alineación donde se establece la condición 
        de parada cuando el score de Smith-Waterman es máximo entre el *$\|\overrightarrow{sB^{n`}}\|$ y *length sentA*. 
        </p>
    </td>
</tr>
</table>


In [6]:
def getOrigSent(text2, offsetB, nextPoint,sentLength):
    posB = text2[offsetB+sentLength:].find('.')
    sentLength += posB+1
    sentB = text2[offsetB:offsetB+sentLength]
    nextPoint = offsetB + sentLength
    return sentB, nextPoint, sentLength

text_orig = open('data/orig/susp/suspicious-document00184.txt').read()

offsetB = 0;nextPoint = 0;sentLength=0

for i in range(text_orig.count('.')):
    sentB, nextPoint, sentLength = getOrigSent(text_orig,offsetB,nextPoint,sentLength)
    print('i',i,'sentB:',sentB)
    print('offsetB:', offsetB, 'nextPoint:', nextPoint, 'sentLength', sentLength)
    print('***************')
    if i == 2: #This is a cuting point see the explanation below
        offsetB = nextPoint
        nextPoint = 0
        sentLength = 0

i 0 sentB: The is a 5 acre (20,000 m ) located in the of , located in .
offsetB: 0 nextPoint: 60 sentLength 60
***************
i 1 sentB: The is a 5 acre (20,000 m ) located in the of , located in . The zoo is operated by the in partnership with the .
offsetB: 0 nextPoint: 113 sentLength 113
***************
i 2 sentB: The is a 5 acre (20,000 m ) located in the of , located in . The zoo is operated by the in partnership with the .Queens Zoo zoo New York City borough Queens Flushing Meadows-Corona Park Wildlife Conservation Society New York City Department of Parks and Recreation
The Zoo opened on , , with the ceremonial ribbon cut by .
offsetB: 0 nextPoint: 322 sentLength 322
***************
i 3 sentB:  The zoo is home mostly to animals native to North America.
offsetB: 322 nextPoint: 381 sentLength 59
***************
i 4 sentB:  The zoo is home mostly to animals native to North America. The Queens Zoo is the only one of the five zoos in New York City to exhibit .
offsetB: 322 nextPoint

**Note:** as you can prove later in the alingment algorithm, when sentA match with sentB the value of the offset parameter is equalized to *nextPoint*. Then the algorithm start to look for the next sentB from the final position (*last* **nextPoint**) of previous sentB. In the above example 322 is at the same time the las nextPoint(& len) of $sentB_1$, and the offset of $sentB_2$, this one start with the string: *The zoo is home mostly...*.

## Jaccard Function

Originally Jaccard distance is evaluated as $(A\cup B- B\cap A)/B\cup A)$, which means the fraction between different labels and similar labels inside A & B sets. Here we implement a variation $Jaccard=(B\cap A)/B\cup A)$, which means the fraction between common terms and the total different terms.

In [7]:
def jaccard(text1,text2):
    sentA1 = re.sub(r'[!"#$%&()\'*+,-/:;<=>?@\\^_`{|}~.\[\]]',' ', text1)
    sentB1 = re.sub(r'[!"#$%&()\'*+,-/:;<=>?@\\^_`{|}~.\[\]]',' ', text2)
    setA = set(sentA1.split())
    setB = set(sentB1.split())
    total_terms = len(sentB1.split())
    return len(setA.intersection(setB))/(total_terms or 0)

# Alignment Pipeline

* The next code print jaccard score between sentence A & B. 
* Then print the maximal score finded for a chunck B given sentence A. 
* Next the main properties of chunck B (offset & length) are printed. 
* Finally for a visual comparation of result sentence A is printed.

**Nota:** To see more details about how process occurs uncomment the *print* orders.

In [18]:
import re

def alignSentences(preproc_text, orig_text):
    sentenceList=[]
    offsetB = 0
    
    norm_orig_text = normalize(orig_text)
    
    #if preproc_text.count('.') > norm_orig_text.count('.'):
    #    raise Exception("Preprocess Error: number of preproc periods most be less or equal than normalize original text periods.")
        
    for i, (sentA, offsetA, lengthA) in enumerate(getSentA(preproc_text)):
        maxScore =-1; score = 0
        prevPoint = 0 #len(sentA)-2
        nextPoint = 0
        iqualScore = 0
        prevFrag=''
        jaccard_measure = 0
        prev_jaccard_measure = 1.0
        k = 0.5
        
        #Sí llegamos a la última oración entonces
        if i == preproc_text.count('.')-1: 
            lengMax = len(norm_orig_text)
            tuple = (i, sentA, offsetB, lengMax)
            sentenceList.append(tuple)
            break
        
        #Sí no es la última oración compara hasta encontrar el score max.
        while(score >= maxScore):
            prev_jaccard_measure = jaccard_measure #only for print reasons
            lengMax = nextPoint
            maxScore = score
            
            #Get original Sentence and prepare it to calc distances
            print('-----from',offsetB,'---- to prevPoint', offsetB+prevPoint)
            sentB, nextPoint, prevPoint = getOrigSent(norm_orig_text, offsetB, nextPoint, prevPoint)
            print('-----to nextPoint',nextPoint)
            print('OrigSent:\n',sentB,'\n')
            print('sentA:',sentA,'\n')
            
            #Calc measures Jaccard
            jaccard_measure = jaccard ( sentA , sentB) #Second measure only to lookfor errors
            print ('jaccard_measure:',jaccard_measure)
            
            score = jaccard_measure
            print('i:',i,'score:',score,'maxScore:',maxScore)
            print('frag-sentA:',sentA[-round(len(sentA)*k):],'\nfrag-sentB:',sentB[-round(len(sentA)*k):],'\n')
            
            #Repeated sentence exception src00014
            if prevFrag == sentB[-round(len(sentA)*k):]:
                #print ('=================Repeated sentence')
                break
            
            #keep the previous fragment to know if the next sent is the same as before. 
            #The algh move forward to the next sentence.
            prevFrag = sentB[-round(len(sentA)*k):] 
            
            #Short sentence exceptions
            if len(sentA) < 14:
                maxScore = score
                lengMax = nextPoint
                break
                
            #Infinite loop exception
            if score == maxScore:
                iqualScore += 1
            if iqualScore == 20:
                break
           
        #Raising error because of normalization troubles
        print('maxScore',maxScore)
        if maxScore < 0.5:
            raise ValueError('A normalization error detected.')
        
        tuple = (i, sentA, offsetB, lengMax)
        sentenceList.append(tuple)
        
        if i > -1:
            print ('Final',i,'\njaccard_measure:',prev_jaccard_measure)
            print('Sentence' ,i, 'score max:',maxScore, 'offsetB:', offsetB, 'lengthB:',lengMax-offsetB,'\n')
            print('sent original:',orig_text[offsetB:lengMax],'\n')
            print('sent process:',sentA)
            print('\n***************')
        
        offsetB = lengMax

    return sentenceList
#suspicious-document00017.txt source-document00534.txt
#text_orig = open('data/orig/susp/suspicious-document00017.txt').read()
#preproc_text = open('data/norm/susp/suspicious-document00017.txt').read()

text_orig = open('data/orig/src/source-document00534.txt').read()
preproc_text = open('data/norm/src/source-document00534.txt').read()

sentenceList = alignSentences(preproc_text,text_orig)

-----from 0 ---- to prevPoint 0
-----to nextPoint 19
OrigSent:
 0 [] edit Vehicles. 

sentA: 0 edit Vehicles . 

jaccard_measure: 1.0
i: 0 score: 1.0 maxScore: 0
frag-sentA: hicles . 
frag-sentB: ehicles. 

-----from 0 ---- to prevPoint 19
-----to nextPoint 48
OrigSent:
 0 [] edit Vehicles.The Maine State polices uses. 

sentA: 0 edit Vehicles . 

jaccard_measure: 0.375
i: 0 score: 0.375 maxScore: 1.0
frag-sentA: hicles . 
frag-sentB: es uses. 

maxScore 1.0
Final 0 
jaccard_measure: 1.0
Sentence 0 score max: 1.0 offsetB: 0 lengthB: 19 

sent original: 0 [] edit Vehicles
 

sent process: 0 edit Vehicles .

***************
-----from 19 ---- to prevPoint 19
-----to nextPoint 48
OrigSent:
 The Maine State polices uses. 

sentA: The Maine State polices uses . 

jaccard_measure: 1.0
i: 1 score: 1.0 maxScore: 0
frag-sentA:  polices uses . 
frag-sentB: e polices uses. 

-----from 19 ---- to prevPoint 48
-----to nextPoint 94
OrigSent:
 The Maine State polices uses.Ford and in both marked and u

-----from 2877 ---- to prevPoint 2877
-----to nextPoint 2923
OrigSent:
 Maine State Police Cruiser - Ford Interceptor. 

sentA: Maine State Police Cruiser Ford Interceptor . 

jaccard_measure: 1.0
i: 36 score: 1.0 maxScore: 0
frag-sentA: ser Ford Interceptor . 
frag-sentB: er - Ford Interceptor. 

-----from 2877 ---- to prevPoint 2923
-----to nextPoint 2968
OrigSent:
 Maine State Police Cruiser - Ford Interceptor.The State Police have many specialized units. 

sentA: Maine State Police Cruiser Ford Interceptor . 

jaccard_measure: 0.46153846153846156
i: 36 score: 0.46153846153846156 maxScore: 1.0
frag-sentA: ser Ford Interceptor . 
frag-sentB: any specialized units. 

maxScore 1.0
Final 36 
jaccard_measure: 1.0
Sentence 36 score max: 1.0 offsetB: 2877 lengthB: 46 

sent original: Maine State Police Cruiser - Ford Interceptor
 

sent process: Maine State Police Cruiser Ford Interceptor .

***************
-----from 2923 ---- to prevPoint 2923
-----to nextPoint 2968
OrigSent:
 The State P

i: 102 score: 1.0 maxScore: 0
frag-sentA: are all IAI certified Scene Technicians and Analysts . 
frag-sentB:  are all IAI certified Scene Technicians and Analysts. 

-----from 8446 ---- to prevPoint 8554
-----to nextPoint 8730
OrigSent:
  242 30673 The ERT is composed of 29 sworn members who are all IAI certified Scene Technicians and Analysts. 306 38774 These members may have individual forensic specialties but are all generalists that maintain a good working knowledge of the various disciplines of forensic science. 

sentA: 242 30673 The ERT is composed of 29 sworn members who are all IAI certified Scene Technicians and Analysts . 

jaccard_measure: 0.4222222222222222
i: 102 score: 0.4222222222222222 maxScore: 1.0
frag-sentA: are all IAI certified Scene Technicians and Analysts . 
frag-sentB: wledge of the various disciplines of forensic science. 

maxScore 1.0
Final 102 
jaccard_measure: 1.0
Sentence 102 score max: 1.0 offsetB: 8446 lengthB: 108 

sent original:  242 30673 The ERT 

### Testing the Alignment result

In [9]:
print ('oración preprocesada:', sentenceList[2][1])
print ('oración original:', text_orig[sentenceList[2][2]:sentenceList[2][3]])

oración preprocesada: Civilians 587 as of 2004 4 .
oración original: Civilians
587 (as of 2004) [4]



## Getting the Tagged Aligned Text of Whole Collection

Here we have again the *file* **pairs** which contains all normalized cases, inside the normalizated collection (../norm), then we need the route to original files and finally the output directory.

Rememberíng the [begining definition](#Aligned_Text_Structure) of <font color='#FA0000'>aligned text structure</font> for future phases.

E.g. open the file *data/aligned/susp/suspicious-document00007.txt*
<i>
<p>    0	According to the legend a dish was make by servants of country kings paella were let to take mixed leftovers from the large dinner home in courtly pots .	0	154
<p>    1	It iseafood believed that the Arabic word woulderives from the paella word which means leftovers .	154	248
<p>    2	Take spanish dish guides Paella probably a a other rich .	248	305
<p>    ...
</i>

$(id_K,normalized-sentence_K,original-offset_{sentence\,K},original-length_{sentence\,K})$

In [10]:
%run scripts/02.2_alignNormalizedCaseList.py norm/norm_pairs norm/src/ norm/susp/ aligned/

The script will use as default the data folder as working directory
suspicious-document00017.txt source-document00534.txt


ValueError: A normalization error detected.

# Conclusions

The objective of this process was accomplished. The obtained transformed text contains the proposed structure. 1361 pairs of text (cases) were obtaibned previously from the 2000 initial normalized list using initial versions. Then we re-aligned thems to show the process of alignment using 02.2_alignNormalizedCaseList.py script. In the final version of this script we get 2000 pair cases thanks to fine dev of func _normalize()_.

Although 2.2 script use the Jaccard alignment algorithm and the 100% of the cases were good aligned after normalization, it is very easy to test other similarity distances or alignment knowleage in a different corpus and obtain different results and performance, based on original_text_corpus particularities.

# Excersices

1. The comparison between alignment text using Smith-Waterman and the algh presented here can show a different performance. Implement a new alignment pipeline using another faster technique and less source code.

2. Make a list of different punctuation situations that can appear in real text provided by pdftotext conversors and program the regular expresion for both sides the normalization process and for normalize function. Construct a test unit with your ideas and finally test them in the same way we did here.

3. Create a different idea to align original and preprocessed sentences. Sugestion, take a look to cross-lingual alignment implemented in nltk.

4. In the script/align.py version of this alignment algorithm The next code was added to the alignment pipeline to improve the alignment of long sentences. Evaluate it and propose the best k to maintain the precision.

        #Optimization for very long sentences alignment
        if len(sentA) > 500:
            k = 0.1
        else: k=0.5
        
5. The pairs list provided for this tutorial(_norm_pairs_) contains 2000 cases in which the Jaccard based aligment makes a perfect score. If this algorithm is tested on all PAN-PC pairs file the result could be very different, some cases will be bad aligned. Make the necessary changes to the algorithm to write in the alignedDocDict file a "False" value if the text is bad aligned. (_Sugestion: study the script/align.py file_)